# Houston 311 Service Requests Data Mining Project

## Group members
Stephen Huang

Levi Villarreal

Joshua Wong

Andrew Young


### Code Imports

In [50]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
import time
import datetime
import zipfile

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
import pickle

### Import data

In [27]:

# Data is from http://www.houstontx.gov/311/
names = ["CASE NUMBER", "SR LOCATION", "COUNTY", "DISTRICT", "NEIGHBORHOOD", "TAX ID", 
         "TRASH QUAD", "RECYCLE QUAD", "TRASH DAY", "HEAVY TRASH DAY", "RECYCLE DAY", 
         "KEY MAP", "MANAGEMENT DISTRICT", "DEPARTMENT", "DIVISION", "SR TYPE", "QUEUE", 
         "SLA", "STATUS", "SR CREATE DATE", "DUE DATE", "DATE CLOSED", "OVERDUE", "TITLE", 
         "x", "y", "LATITUDE", "LONGITUDE", "CHANNEL TYPE"]

with zipfile.ZipFile("311-Public-Data-Extract-2018-clean.txt.zip","r") as zip_ref:
    zip_ref.extractall("")

data = pd.read_csv('311-Public-Data-Extract-2018-clean.txt', sep="|", header=None, names=names)
print(data.shape)
data.head()

/Users/levivillarreal/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,22,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(399953, 29)


CASE NUMBER                                        SR LOCATION  \
0            CASE NUMBER                                        SR LOCATION   
1           101002866096  Intersection 13300 HEMPSTEAD RD&9100 PINEMONT ...   
2  12202635-101002866097                  2803 W BELLFORT, HOUSTON TX 77054   
3  12202636-101002866098                      2115 HUTTON, HOUSTON TX 77026   
4  12202637-101002866099              14806 STEEPLE CHASE, HOUSTON TX 77489   

      COUNTY  DISTRICT                    NEIGHBORHOOD            TAX ID  \
0     COUNTY  DISTRICT                    NEIGHBORHOOD            TAX ID   
1        NaN         A  FAIRBANKS / NORTHWEST CROSSING               NaN   
2        NaN         K                      SOUTH MAIN               NaN   
3     HARRIS         B              GREATER FIFTH WARD     0170560000001   
4  FORT BEND         K               FORT BEND HOUSTON  9550030090110907   

   TRASH QUAD  RECYCLE QUAD  TRASH DAY  HEAVY TRASH DAY  ...  \
0  TRASH QUAD  RECYCLE QUAD  TRASH DAY  HEAVY TRASH DAY  ...   
1         NaN           NaN        NaN              NaN  ...   
2         NaN           NaN        NaN              NaN  ...   
3          NE            NW     FRIDAY     1st Thursday  ...   
4          SW            SW   THURSDAY     3rd Thursday  ...   

        SR CREATE DATE             DUE DATE          DATE CLOSED  OVERDUE  \
0       SR CREATE DATE             DUE DATE          DATE CLOSED  OVERDUE   
1  2018-01-01 00:38:05  2018-01-02 00:38:05  2018-01-01 01:31:49    -0.96   
2  2018-01-01 00:41:49  2018-01-11 00:41:49  2018-01-01 05:45:02    -9.79   
3  2018-01-01 00:45:07  2018-01-03 00:45:07  2018-01-01 09:55:02    -1.62   
4  2018-01-01 00:51:20  2018-01-06 00:51:20  2018-01-01 16:10:02    -4.36   

                                     TITLE                    x  \
0                                    Title                    x   
1  Traffic Signal Maintenance-101002866096  3073146.72528000000   
2                  Water Leak-101002866097  3104254.38025000000   
3               Water Service-101002866098  3131779.00434000000   
4            Water Main Valve-101002866099  3079284.91871000000   

                      y     LATITUDE     LONGITUDE  CHANNEL TYPE  
0                     y     LATITUDE     LONGITUDE  Channel Type  
1  13869806.76417000000          NaN           NaN      Voice In  
2  13809501.20958000000  29.67268442  -95.42283905      Voice In  
3  13849821.40912000000   29.7811689  -95.33222248      Voice In  
4  13786008.44784000000  29.61018299  -95.50364663      Voice In  

[5 rows x 29 columns]

## Dimensionality Reduction
Right off the bat, we can see that there are many features that do not matter at all to our project.
We can go ahead and get rid of these to help prevent against the curse of dimensionality.

In [28]:
data = data.drop(columns=['SR LOCATION', 'CASE NUMBER', 'TRASH QUAD', 'RECYCLE QUAD', 'TRASH DAY', 'HEAVY TRASH DAY', 'RECYCLE DAY', 'TAX ID', 'KEY MAP', 'DUE DATE', 'DATE CLOSED', 'TITLE', 'x', 'y', 'LONGITUDE', 'LATITUDE'])
print(data.shape)

data.head()

(399953, 13)


COUNTY  DISTRICT                    NEIGHBORHOOD  MANAGEMENT DISTRICT  \
0     COUNTY  DISTRICT                    NEIGHBORHOOD  MANAGEMENT DISTRICT   
1        NaN         A  FAIRBANKS / NORTHWEST CROSSING     Spring Branch MD   
2        NaN         K                      SOUTH MAIN                  NaN   
3     HARRIS         B              GREATER FIFTH WARD                  NaN   
4  FORT BEND         K               FORT BEND HOUSTON                  NaN   

                     DEPARTMENT             DIVISION  \
0                    DEPARTMENT             DIVISION   
1  PWE Public Works Engineering   Traffic Operations   
2  PWE Public Works Engineering  PU Public Utilities   
3  PWE Public Works Engineering  PU Public Utilities   
4  PWE Public Works Engineering  PU Public Utilities   

                      SR TYPE                  QUEUE  SLA  STATUS  \
0                     SR TYPE                  QUEUE  SLA  STATUS   
1  Traffic Signal Maintenance  TT_SignalMainDispatch    1  Closed   
2                  Water Leak               PU_Water   10  Closed   
3               Water Service               PU_Water    2  Closed   
4            Water Main Valve               PU_Water    5  Closed   

        SR CREATE DATE  OVERDUE  CHANNEL TYPE  
0       SR CREATE DATE  OVERDUE  Channel Type  
1  2018-01-01 00:38:05    -0.96      Voice In  
2  2018-01-01 00:41:49    -9.79      Voice In  
3  2018-01-01 00:45:07    -1.62      Voice In  
4  2018-01-01 00:51:20    -4.36      Voice In

### Reasoning behind dropping the attributes we did

SR Location - Same reasoning as case number.

Case number - This was dropped because the case number is a unique identifier assigned to every 311 request, and thus could not be used to classify anything.

Trash Quad - There are requests that are not related to trash collection. Those requests might be misclassified because they are in a different trash quad as other requests in the dataset.

Recycle Quad - Same as Trash Quad

Trash Day - Same as Trash Quad

Recycle day - Same as Trash Quad

Heavy Trash Day - Same as Trash Quad
 
Tax ID - This ID is unique to the property that requested it.
 
Key Map - This attribute is used in the cities internal system to keep track of requests and does not have any actual relation to the request
 
Due Date - We are interested in predicting overdueness, and that it its own seperate column.
 
Date Closed - Same reasoning as due date.
 
Title - This is a another unique identifier assigned to every 311 request, and thus could not be used to classify anything.
 
x - We already have latitude, and this attribute an approximation of latitude.
 
y - We already have longitude, and this attribute an approximation of longitude.

latitude - This is too speficific for our needs, and we have other location metrics.
 
longitude - same as latitude

## Data Cleaning

Drop all service requests that are not closed

In [29]:
oldNumRows = data.shape[0]
data = data[data.STATUS == 'Closed']
data = data.drop(["STATUS"], axis=1)

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  181


Normalize all counties and drop rows where county is unknown

In [30]:
print('Counties:')
print(data['COUNTY'].value_counts())
print()

oldNumRows = data.shape[0]
data['COUNTY'] = data['COUNTY'].replace(['HARRIS', 'FORT BEND', 'MONTGOMERY'], ['Harris County', 'Fort Bend County', 'Montgomery County'])
data = data[data.COUNTY != 'Unknown']

print('Counties:')
print(data['COUNTY'].value_counts())
print()
print("Number of rows with COUNTY == 'Unknown' dropped: ", oldNumRows - data.shape[0])

Counties:
HARRIS               322110
Harris County         36673
FORT BEND              6868
Unknown                3201
Fort Bend County        379
MONTGOMERY              185
Montgomery County       120
Galveston County         13
Name: COUNTY, dtype: int64

Counties:
Harris County        358783
Fort Bend County       7247
Montgomery County       305
Galveston County         13
Name: COUNTY, dtype: int64

Number of rows with COUNTY == 'Unknown' dropped:  3201


Drop rows where district is unknown

In [31]:
oldNumRows = data.shape[0]
data = data[data.DISTRICT != 'Unknown']

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  552


Normalize all districts

In [32]:
data['MANAGEMENT DISTRICT'] = data['MANAGEMENT DISTRICT'].replace(['HCID #3 TRACT 19 (Upper Kirby)', 'HCID #3 TRACT 47 (Upper Kirby)', 'East End MD', 'Greater Northside', 'Sharpstown'], ['HCID #3 (Upper Kirby)', 'HCID #3 (Upper Kirby)', 'East End', 'Greater Northside MD', 'Sharpstown MD'])
print('Management Districts:')
print(data['MANAGEMENT DISTRICT'].value_counts().sort_index())

Management Districts:
Airline IMPROVEMENT DISTRICT                   23
Airline PID                                     6
Aldin North Expansion Tract2                    6
Aldine North Expansion Tract1                  18
Aldine PID                                    232
Baybrook MD                                   110
East Downtown MD                             1743
East End                                    16767
FB CAD #6                                       6
Five Corners HCID #10B                      11645
Five Corners Improvement Dist(HCID #10B)       59
Greater Greenspoint MD                       1834
Greater Northside MD                        32037
Greater Southeast MD                        12224
HCID #10-A                                  14007
HCID #12                                        6
HCID #16                                        4
HCID #3 (Upper Kirby)                        1743
HCID #4 (Energy Corridor)                     956
HCID #5 (Brays Oaks)        

Drop all rows with unknown channel types

In [33]:
oldNumRows = data.shape[0]
data = data[data['CHANNEL TYPE'] != 'Unknown']

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  20590


Drop all rows with null values

In [34]:
oldNumRows = data.shape[0]
data = data.dropna()

print("Number of rows dropped: ", oldNumRows - data.shape[0])

Number of rows dropped:  226480


## Feature Transformation

Currently, the request date would be draw conclusions from, so we transformed the feature to instead be the month in which the service request was created, to account seasonal differences in service request time.

In [35]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
data['SR CREATE DATE'] = data['SR CREATE DATE'].apply(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data['SR CREATE DATE'] = data['SR CREATE DATE'].apply(lambda x : months[x.month-1])

data = data.rename(columns={'SR CREATE DATE': 'SR MONTH'})
data.head()

COUNTY DISTRICT         NEIGHBORHOOD     MANAGEMENT DISTRICT  \
10  Harris County        I   GREATER HOBBY AREA                 HCID #9   
16  Harris County        D            SUNNYSIDE  Five Corners HCID #10B   
17  Harris County        A  SPRING BRANCH NORTH        Spring Branch MD   
20  Harris County        J           SHARPSTOWN           Sharpstown MD   
22  Harris County        F                ALIEF        International MD   

                      DEPARTMENT                  DIVISION  \
10  PWE Public Works Engineering       Street and Drainage   
16      NS Neighborhood Services            Investigations   
17    SWM Solid Waste Management  General Support Services   
20  PWE Public Works Engineering       PU Public Utilities   
22    SWM Solid Waste Management               Collections   

                 SR TYPE              QUEUE  SLA SR MONTH OVERDUE CHANNEL TYPE  
10         Street Hazard    ROWM_StreetMain   10  January   -8.83     Voice In  
16  Nuisance On Property        NS_Dispatch  180  January  134.19     Voice In  
17    Dumpster Complaint     SWM_Complaints   21  January   -0.70     Voice In  
20      Sewer Wastewater      PU_WasteWater    1  January   -0.96     Voice In  
22     Container Problem  SWM_CollectionsSW   10  January   -7.64     Voice In

Currently, the overdue is continuous, which makes it hard to use in classification algorithms. To combat this, we want to bin the values, so that there are only a few possible values.

The values we want bin into are on time tasks (negative values), tasks done within a week past due date (0 < x < 7), tasks done within a month past due date (7 <= x < 30), tasks done over a month past due date (x >= 30).

In [36]:
def bin_overdue (val):
    val = float(val)
    
    if val < 0:
        return "On Time"
    elif val < 7:
        return "Week"
    elif val < 30:
        return "Month"
    else:
        return "More"
    
data["OVERDUE"] = data["OVERDUE"].apply(bin_overdue)
    
print(data['OVERDUE'].value_counts().sort_index())
data.head()

Month       14171
More         9399
On Time    107013
Week        18366
Name: OVERDUE, dtype: int64


COUNTY DISTRICT         NEIGHBORHOOD     MANAGEMENT DISTRICT  \
10  Harris County        I   GREATER HOBBY AREA                 HCID #9   
16  Harris County        D            SUNNYSIDE  Five Corners HCID #10B   
17  Harris County        A  SPRING BRANCH NORTH        Spring Branch MD   
20  Harris County        J           SHARPSTOWN           Sharpstown MD   
22  Harris County        F                ALIEF        International MD   

                      DEPARTMENT                  DIVISION  \
10  PWE Public Works Engineering       Street and Drainage   
16      NS Neighborhood Services            Investigations   
17    SWM Solid Waste Management  General Support Services   
20  PWE Public Works Engineering       PU Public Utilities   
22    SWM Solid Waste Management               Collections   

                 SR TYPE              QUEUE  SLA SR MONTH  OVERDUE  \
10         Street Hazard    ROWM_StreetMain   10  January  On Time   
16  Nuisance On Property        NS_Dispatch  180  January     More   
17    Dumpster Complaint     SWM_Complaints   21  January  On Time   
20      Sewer Wastewater      PU_WasteWater    1  January  On Time   
22     Container Problem  SWM_CollectionsSW   10  January  On Time   

   CHANNEL TYPE  
10     Voice In  
16     Voice In  
17     Voice In  
20     Voice In  
22     Voice In

## Export data to a CSV

In [37]:
data.to_csv('houston-311-sanitized.csv', index=False)

## Read in Sanitized Data

In [38]:
data = pd.read_csv('houston-311-sanitized.csv', sep=",", header='infer')
print(data.shape)
data.head()

(148949, 12)


COUNTY DISTRICT         NEIGHBORHOOD     MANAGEMENT DISTRICT  \
0  Harris County        I   GREATER HOBBY AREA                 HCID #9   
1  Harris County        D            SUNNYSIDE  Five Corners HCID #10B   
2  Harris County        A  SPRING BRANCH NORTH        Spring Branch MD   
3  Harris County        J           SHARPSTOWN           Sharpstown MD   
4  Harris County        F                ALIEF        International MD   

                     DEPARTMENT                  DIVISION  \
0  PWE Public Works Engineering       Street and Drainage   
1      NS Neighborhood Services            Investigations   
2    SWM Solid Waste Management  General Support Services   
3  PWE Public Works Engineering       PU Public Utilities   
4    SWM Solid Waste Management               Collections   

                SR TYPE              QUEUE    SLA SR MONTH  OVERDUE  \
0         Street Hazard    ROWM_StreetMain   10.0  January  On Time   
1  Nuisance On Property        NS_Dispatch  180.0  January     More   
2    Dumpster Complaint     SWM_Complaints   21.0  January  On Time   
3      Sewer Wastewater      PU_WasteWater    1.0  January  On Time   
4     Container Problem  SWM_CollectionsSW   10.0  January  On Time   

  CHANNEL TYPE  
0     Voice In  
1     Voice In  
2     Voice In  
3     Voice In  
4     Voice In

## Prep the data for modeling

Split the data into features and labels. The label is the overdueness of the 311 request, which we binned earlier.

Because of the way that sklearn handles catagorical data, we had to encode all of the features so that they could be processed correctly.

Also, we downsample the data so that the models will run quicker. We use `train_test_split` here to ensure that the downsampling is down without affecting the class balance.

In [76]:
num_data = data.copy()
cat_columns = ["COUNTY", "DISTRICT", "NEIGHBORHOOD", "MANAGEMENT DISTRICT", "DEPARTMENT", "DIVISION",
              "SR TYPE", "QUEUE", "SR MONTH", "CHANNEL TYPE"]

for c in cat_columns:
    prev_column = num_data[c]
    # Transform catagorical data to one that sklearn can understand
    encoder = preprocessing.LabelEncoder()
    num_data[c] = encoder.fit_transform(prev_column)

print(num_data.head())
train, test = train_test_split(num_data, train_size = 0.05, test_size = 0.05)

   COUNTY  DISTRICT  NEIGHBORHOOD  MANAGEMENT DISTRICT  DEPARTMENT  DIVISION  \
0       1         8            28                   16          14        27   
1       1         3            70                    4          11        16   
2       1         0            68                   30          15        13   
3       1         9            61                   29          14        21   
4       1         5             3                   20          15         1   

   SR TYPE  QUEUE    SLA  SR MONTH  OVERDUE  CHANNEL TYPE  
0      115     60   10.0         4  On Time             4  
1       80     35  180.0         4     More             4  
2       26     67   21.0         4  On Time             4  
3      108     53    1.0         4  On Time             4  
4       17     65   10.0         4  On Time             4  


In [77]:
down_data = train

labels = down_data['OVERDUE']
df_features = down_data.drop(['OVERDUE'], axis=1)
print(labels.value_counts().sort_index())

Month       66
More        53
On Time    530
Week        95
Name: OVERDUE, dtype: int64


## Decision Trees

In [80]:
features_train, features_test, labels_train, labels_test = train_test_split(df_features, labels, test_size = 0.2)
print('Training set size:',len(labels_train))
print('Test set size:    ',len(labels_test))
clf = tree.DecisionTreeClassifier()
clf = clf.fit(features_train, labels_train)
predict_labels_test = clf.predict(features_test)
print('Accuracy of decision tree classifier:',accuracy_score(labels_test, predict_labels_test))

Training set size: 595
Test set size:     149
Accuracy of decision tree classifier: 0.6577181208053692


## Support Vector Machines (SVM)

In [81]:
pipe = Pipeline(steps=[('scaler',StandardScaler()),('reduce_dim',PCA()),('clf',SVC(gamma='scale'))])
param_grid = {
    'reduce_dim__n_components': list(range(5, 11)),
    'clf__kernel':('linear','rbf','poly')
}
accuracies = cross_val_score(GridSearchCV(pipe,param_grid,cv=5,scoring='accuracy', iid=True).fit(df_features,labels), df_features, labels, cv=10)
print('Official accuracy:',np.mean(accuracies))

KeyboardInterrupt: 

## Neural Networks (NN)

In [50]:
from sklearn.neural_network import MLPClassifier
pipe = Pipeline(steps=[('scaler',StandardScaler()),('clf',MLPClassifier())])
param_grid = {
    'clf__hidden_layer_sizes':((30,),(40,),(50,),(60,)),
    'clf__activation':('logistic','tanh','relu')
}
accuracies = cross_val_score(GridSearchCV(pipe,param_grid,cv=5,scoring='accuracy', iid=True).fit(df_features,labels), df_features, labels, cv=5)
print('Official accuracy:',np.mean(accuracies))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

...

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, Converge

/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Official accuracy: 0.7409994182223418


/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## k-Nearest Neighbor (KNN)

In [83]:
from sklearn.neighbors import KNeighborsClassifier
param_grid = {
    'knn__n_neighbors': list(range(1, 25))
}
pipe = Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA()), ('knn',KNeighborsClassifier(n_neighbors=7))])
accuracies = cross_val_score(GridSearchCV(pipe,param_grid,cv=5,scoring='accuracy', iid=True).fit(df_features,labels), df_features, labels, cv=5)
print('Average accuracy:',np.mean(accuracies))

Average accuracy: 0.6948884454924722


## Naive Bayes
Naive Bayes is not a good option for this dataset due to correlation between attributes. For example, requests from the same neighborhood would be from the same county.

In [52]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
accuracies = cross_val_score(clf,df_features,labels,cv=10)
print('Average accuracy:',np.mean(accuracies))

Average accuracy: 0.5812010016862861


## AdaBoost

In [84]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=150)
accuracies = cross_val_score(clf, df_features, labels, cv=5)
print('Average accuracy:',np.mean(accuracies))

Average accuracy: 0.6007634077030051


## Random Forest

In [86]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)
param_grid = {
    'max_depth':list(range(35,55)),
    'min_samples_leaf':(8,10,12),
    'max_features':('sqrt','log2')
}
accuracies = cross_val_score(GridSearchCV(clf,param_grid,cv=5,scoring='accuracy', iid=True).fit(df_features,labels), df_features, labels, cv=5)
print('Official accuracy:',np.mean(accuracies))

/Users/levivillarreal/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/levivillarreal/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/levivillarreal/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/levivillarreal/opt/anaco

Official accuracy: 0.7150862809117842


/Users/levivillarreal/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
